# Keras MiniVGGNet example on cifar10

In [ ]:
!git clone https://github.com/AlbertoRomanRamosRodriguez/keras_example.git

In [ ]:
import matplotlib
matplotlib.use('Agg')

from keras_example.nn.conv.minivggnet import MiniVGGNet

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import load_model

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("[INFO] accessing MNIST...")
((trainData, trainLabels),(testData, testLabels))=cifar10.load_data()

trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0

le = LabelBinarizer()
trainLabels = le.fit_transform(trainLabels)
testLabels = le.transform(testLabels)

labelNames = ["airplane", "automobile", "bird", "cat", "deer",
"dog", "frog", "horse", "ship", "truck"]

In [ ]:
print("[INFO] compiling model...")
EPOCHS = 40
LR = 0.01

opt = SGD(lr=LR, decay = LR/EPOCHS, momentum=0.9, nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=len(labelNames))
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = model.fit(trainData, trainLabels,
validation_data=(testData, testLabels), batch_size=64,
epochs=EPOCHS, verbose=1)

In [ ]:
def evaluate_model(model_name:str):

  model = load_model(f'./models/{model_name}')
  print(f"[INFO] evaluating {model_name}...")
  predictions = model.predict(testData, batch_size=32)
  print(classification_report(testLabels.argmax(axis=1), 
    predictions.argmax(axis=1)))

  plt.style.use('ggplot')
  plt.figure()

  plt.plot(np.arange(0,EPOCHS), H.history['loss'], label='train_loss')
  plt.plot(np.arange(0,EPOCHS), H.history['val_loss'], label='val_loss')
  plt.plot(np.arange(0,EPOCHS), H.history['accuracy'], label='train_acc')
  plt.plot(np.arange(0,EPOCHS), H.history['val_accuracy'], label='val_acc')
  plt.title("Training Loss and Accuracy")
  plt.xlabel("Epoch #")
  plt.xlabel("Loss/Accuracy")
  plt.legend()
  plt.show()

In [ ]:
from datetime import datetime

date = datetime.today().strftime("%m_%d_%Y_%H_%M_%S")

model.save(f"./models/model_mnist_lenet_{date}.hdf5")

In [ ]:
for m in os.listdir('./models/'):
    evaluate_model(m)